# Proyecto No. 1 Data Science
Universidad del Valle de Guatemala
Segundo Semestre 2021

Andrés Say, 19705
Ayleen Rubio, 19003
Diego Alegría, 15171
Mario Sarmientos, 17055

## Importación de librerías

In [1]:
import pandas as pd
import numpy as np
import os

ModuleNotFoundError: No module named 'pandas'

## Lectura de directorio y unificación de Data Frame

In [ ]:
files=os.listdir('.\EstablecimientosMINEDUC')
df=pd.read_csv('.\EstablecimientosMINEDUC\\'+files[0],encoding = "ISO-8859-1",header=1,skipfooter=2,engine='python')
df=pd.DataFrame(columns=df.columns)
for i in files:
    temp=(pd.read_csv('.\EstablecimientosMINEDUC\\'+i,encoding = "ISO-8859-1",header=1,skipfooter=2,engine='python'))
    
    temp.loc[temp.DISTRITO=='','DISTRITO']=np.nan
    temp.loc[temp.DEPARTAMENTO=='','DEPARTAMENTO']=np.nan
    temp.loc[temp.MUNICIPIO=='','MUNICIPIO']=np.nan
    temp.loc[temp.DIRECCION=='nan','DIRECCION']=np.nan
    temp.loc[temp.TELEFONO=='nan','TELEFONO']=np.nan
    temp.loc[temp.ESTABLECIMIENTO=='nan','ESTABLECIMIENTO']=np.nan
    temp.loc[temp.DIRECCION=='','DIRECCION']=np.nan
    temp.loc[temp.TELEFONO=='','TELEFONO']=np.nan
    
    
    temp.DEPARTAMENTO=temp.DEPARTAMENTO.fillna(method='ffill')
    
    df=df.append(temp,ignore_index=True)
    
df

## Lista de variables (Columnas)

In [ ]:
display(df.columns.tolist())

## Tipos de variables

In [ ]:
df.dtypes

## Análisis para variable Código

#### Revisión de formato

In [ ]:
display(df.CODIGO.unique())

#### Revisión de Nulls

In [ ]:
# Utilizando el criterio de pandas
display(df.loc[df.CODIGO.isna(),:])

# Utilizando el criterio de la palabra nan

display(df.loc[df.CODIGO=='nan',:])

#### Revisión de duplicidad

In [ ]:
a=len(df.CODIGO.value_counts())
b=len(df)

display(b-a)

#No existe duplicidad


Debido a que es un código que esta compuesto de la siguiente forma:

$$##-##-####-##$$ 

Donde # es un númeor entero del 0 al 9, debemos de revisar si todos tienen los números completos o existen casos donde no se cumpla el estándar antes mencionado. 

No se despliegan todos los valores puesto que cada código es único y no representa ningún tipo de relación con otros establecimientos más si presenta una relación con las otras variables de ubicación como departamento y municipio.


In [ ]:
code=df.CODIGO.str.split('-',expand=True).copy()
code

#### Revisión por partes del código de identificación

In [ ]:
display(code.loc[code[0].isna(),:])
display(code.loc[code[1].isna(),:])
display(code.loc[code[2].isna(),:])
display(code.loc[code[3].isna(),:])

Podemos observar que todo se ve de forma correcta

## Análisis para variable distrito

#### Revisión de Formato

In [ ]:
display(df.DISTRITO.unique().tolist()[0:20])

#### Revisión de Nulls

In [ ]:
display(df.loc[df.DISTRITO.isna(),:])

#### Revisión de códigos únicos

In [ ]:
df.DISTRITO.value_counts()


Debido a que es un Distrito que esta compuesto de la siguiente forma:

$$##-###$$ 

Donde # es un númeor entero del 0 al 9, debemos de revisar si todos tienen los números completos o existen casos donde no se cumpla el estándar antes mencionado. 

No se despliegan todos los valores puesto que cada código es único para el distrito


In [ ]:
distrito=df.DISTRITO.str.split('-',expand=True)

#### Revisión por partes del código de identificación

In [ ]:
print('Para primera parte del código de identificación del distrito')
display(distrito.loc[(distrito[0].isna())|(distrito[0]==''),:])
print('Para segunda parte del código de identificación del distrito')
display(distrito.loc[(distrito[1].isna())|(distrito[1]==''),:])

Podemos notar que tenemos 212 establecimientos que no tienen un código de distrito en la primera parte del código y en la segunda tenemos 245 (los 212 anteriores + 33 de establecimientos que solo poseen la primera mitad del código de distrito)

#### Porcentaje de datos faltantes a corregir

In [ ]:
a=len(distrito.loc[(distrito[1].isna())|(distrito[1]==''),:])
b=len(df)

print(str(round((a/b)*100,2))+' %')

#### Estrategia para corrección

Se plantea una función en donde se busca la coincidencia más cercana con tres criterios:
    
    - Departamento
    - Municipio
    - Región Departamental
    
Y se sustituye el espacio vacío utilizando programación funcional

In [ ]:
def corregir_distrito(df,departamento,municipio,departamental):
    
    temp=df.loc[(df.DEPARTAMENTO==departamento)&(df.MUNICIPIO==municipio)&(df.DEPARTAMENTAL==departamental),'DISTRITO']
    return temp.value_counts()[0]

try:
    df.loc[(df.DISTRITO.isna())|(df.DISTRITO.str.split('-',expand=True)[1]==''),'DISTRITO'] = df.loc[(df.DISTRITO.isna())|(df.DISTRITO.str.split('-',expand=True)[1]==''),:].apply(lambda row : corregir_distrito(df,row['DEPARTAMENTO'],row['MUNICIPIO'],row['DEPARTAMENTAL']), axis = 1)
except:
    print('Todos los códigos ya han sido restaurados')

## Análisis para variable Departamento

#### Revisión de Formato

In [ ]:
display(df.DEPARTAMENTO.unique())

#### Revisión de Nulls

In [ ]:
display(df.loc[df.DEPARTAMENTO.isna(),:])

#### Formato a Proper Case

In [ ]:
df.DEPARTAMENTO=df.DEPARTAMENTO.str.title()

#### Conteo de establecimientos por departamento

In [ ]:
df.DEPARTAMENTO.value_counts()

## Análisis para variable Municipio

#### Revisión de Formato

In [ ]:
# Formato de datos
display(df.MUNICIPIO.unique().tolist()[0:10])

municipios=len(df.loc[~((df.DEPARTAMENTO=='Ciudad Capital')|(df.MUNICIPIO.str.contains('ZONA'))),'MUNICIPIO'].unique())
zonas=len(df.loc[(df.DEPARTAMENTO=='Ciudad Capital')|(df.MUNICIPIO.str.contains('ZONA')),'MUNICIPIO'].unique())

print('Cantidad de municipios con establecimientos en la base de datos: '+str(municipios))
print('Cantidad de zonas de la ciudad capital con establecimientos en la base de datos: '+str(zonas))

#### Revisión de Nulls

In [ ]:
display(df.loc[df.MUNICIPIO.isna(),:])

#### Formato a Proper Case

In [ ]:
df.MUNICIPIO=df.MUNICIPIO.str.title()

#### Eliminación de espacios en blanco no necesarios

In [ ]:
df.MUNICIPIO=df.MUNICIPIO.str.strip()

#### Eliminación de acentos para estandarización

In [ ]:
df.MUNICIPIO=df.MUNICIPIO.str.replace('á','a')
df.MUNICIPIO=df.MUNICIPIO.str.replace('é','e')
df.MUNICIPIO=df.MUNICIPIO.str.replace('í','i')
df.MUNICIPIO=df.MUNICIPIO.str.replace('ó','o')
df.MUNICIPIO=df.MUNICIPIO.str.replace('ú','u')

#### Conteo de establecimientos por municipio o zona de la Ciudad Capital

In [ ]:
df.MUNICIPIO.value_counts()

## Análisis para variable Director

#### Revisión de Formato

In [ ]:
# Formato de datos
display(df.DIRECTOR.unique().tolist()[0:10])


#### Revisión de Nulls

In [ ]:
display(df.loc[df.DIRECTOR.isna(),:].head())
a=len(df.loc[df.DIRECTOR.isna(),:])
b=len(df)
print('Cantidad de establecimientos sin director: '+str(a)+' siendo el '+str(round((a/b)*100,2))+' % de toda la base de datos')

#### Formato a Proper Case 

In [ ]:
df.DIRECTOR=df.DIRECTOR.str.title()

#### Eliminación de espacios en blanco no necesarios

In [ ]:
df.DIRECTOR=df.DIRECTOR.str.strip()

#### Eliminación de acentos para estandarización

In [ ]:
df.DIRECTOR=df.DIRECTOR.str.replace('á','a')
df.DIRECTOR=df.DIRECTOR.str.replace('é','e')
df.DIRECTOR=df.DIRECTOR.str.replace('í','i')
df.DIRECTOR=df.DIRECTOR.str.replace('ó','o')
df.DIRECTOR=df.DIRECTOR.str.replace('ú','u')

#### Se agrega 'No hay dirección registrada' a los datos faltantes

In [ ]:
df.loc[df.DIRECTOR.isna(),'DIRECTOR']='No hay director registrado'
df.loc[df.DIRECTOR.str.contains('-'),'DIRECTOR']='No hay director registrado'
df.loc[df.DIRECTOR=='.','DIRECTOR']='No hay director registrado'


display(df.loc[df.DIRECTOR=='No hay director registrado',:].head())

#### Conteo de establecimientos por director

In [ ]:
df.DIRECTOR.value_counts()

## Análisis para variable Nivel

No hay necesidad de transformaciones, solo se coloca el title case

#### Revisión de Nulls

In [ ]:
display(df.loc[df.NIVEL.isna(),:].head())
a=len(df.loc[df.NIVEL.isna(),:])
b=len(df)
print('Cantidad de establecimientos sin NIVEL: '+str(a)+' siendo el '+str(round((a/b)*100,2))+' % de toda la base de datos')

#### Conteo de variantes

In [ ]:
df.NIVEL.value_counts()

In [ ]:
df.NIVEL=df.NIVEL.str.title()
df.NIVEL

## Análisis para variable Sector
No hay necesidad de transformaciones, solo se coloca el title case

#### Revisión de Nulls

In [ ]:
display(df.loc[df.SECTOR.isna(),:].head())
a=len(df.loc[df.SECTOR.isna(),:])
b=len(df)
print('Cantidad de establecimientos sin SECTOR: '+str(a)+' siendo el '+str(round((a/b)*100,2))+' % de toda la base de datos')

#### Conteo de variantes

In [ ]:
df.SECTOR.value_counts()

In [ ]:
df.SECTOR=df.SECTOR.str.title()
df.SECTOR

## Análisis para variable Área
No hay necesidad de transformaciones, solo se coloca el title case

#### Revisión de Nulls

#### Conteo de variantes

In [ ]:
df.AREA.value_counts()

In [ ]:
df.AREA=df.AREA.str.title()
df.AREA

## Análisis para variable Status
No hay necesidad de transformaciones, solo se coloca el title case

#### Revisión de Nulls

In [ ]:
display(df.loc[df.STATUS.isna(),:].head())
a=len(df.loc[df.STATUS.isna(),:])
b=len(df)
print('Cantidad de establecimientos sin STATUS: '+str(a)+' siendo el '+str(round((a/b)*100,2))+' % de toda la base de datos')

#### Conteo de variantes

In [ ]:
df.STATUS.value_counts()

In [ ]:
df.STATUS=df.STATUS.str.title()
df.STATUS